# Vector Stores and Retrievers

This tutorial will cover LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data from vector databases and other soures for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of Retrieval-Augmented Generation (RAG).

### Concepts

This guide focuses on retrieval of text data. We will cover the following concepts:

- Documents;
- Vector Stores;
- Retrievers.

### Setup

We're expected to be working in a jupyter notebook. We also need the `langchain`, `langchain-chroma`, and `langchain-openai` packages (oh joy):

```ps
conda install langchain langchain-chroma langchain-openai -c conda-forge
```

We already have `langchain`, `langchain-chroma` will not install from conda-forge. I'm not sure that we'll be using the `langchain-openai` package, so we'll hold off on installing it until we know we need it. I'll pip `langchain-chroma` for now.

```ps
pip install langchain-chroma
```

### Documents

LangChain implements a [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- `page_content`: a string representing the content;
- `metadata`: a dictionary containing arbitrary metadata.

The `metadata` attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual `Document` object often represents a chunk of a larger document.

Let's generate some sample documents:

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

Five documents, three sources.

## Vector Stores

Vector search is a common way to store and search over unstructured data (such as unstructured text). The idea is to store numeric vectors that are associated with the text. Given a query, we can [embed](https://python.langchain.com/v0.2/docs/concepts/#embedding-models) it as a vector of the same dimension and use vector similarity metrics to identify related data in the store.

LangChain [VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) objects contain methods for adding text and `Document` objects to the store, and querying them using various similarity metrics. They are often initialized with [embedding](https://python.langchain.com/v0.2/docs/how_to/embed_text/) models, which determine how text data is translated to numeric vectors.

LangChain includes a suite of [integrations](https://python.langchain.com/v0.2/docs/integrations/vectorstores/) with different vector store technologies. Some vector stores are hosted by a provider (e.g., various cloud providers) and require specific credentials to use; some (such as [Postgres](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pgvector/)) run in separate infrastructure that can be run locally or via a third-party; others can run in-memory for lightweight workloads. Here we will demonstrate usage of LangChain VectorStores using [Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/), which includes an in-memory implementation.

To instantiate a vector store, we often need to provide an [embedding](https://python.langchain.com/v0.2/docs/how_to/embed_text/) model to specify how text should be converted to a numeric vector. Here we will deviate from the tutorial a bit. They use the [OpenAI embeddings](https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/). We're going to be switching from even what we were doing yesterday and using Mistralv0.3, as it natively supports function calling. We can use `OllamaEmbeddings` to ask the model for the embeddings.

In [4]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OllamaEmbeddings(
        model="mistral:v0.3"
    ),
)

Calling .from_documents here will add the documents to the vector store. [VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) implements methods for adding documents that can also be called after the object is instantiated. Most implementations will allow you to connect to an existing vector store--e.g., by providing a client, index name, or other information. See the documentation for a specific [integration](https://python.langchain.com/v0.2/docs/integrations/vectorstores/) for more detail.

Once we've instantiated a `VectorStore` that contains documents we can query it. `VectorStore` includes methods for querying:

- Synchronously and asynchronously;
- By string query and by vector;
- With and without returning similarity scores;
- By similarity and [maximum marginal relevance](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.max_marginal_relevance_search).

The methods will generally include a lost of [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) objects in their outputs.

### Examples

Return documents based on similarity to a string query:

In [8]:
vectorstore.similarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

Async query:

In [6]:
await vectorstore.asimilarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

Return scores:

Note that providers implement different scores. In our example Chroma returns a distance metric that should vary inversely with similarity.

In [9]:
vectorstore.similarity_search_with_score("cat")

[(Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  211890.859375),
 (Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  211890.859375),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  216063.953125),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  216063.953125)]

Return documents based on similarity to an embedded query:

In [5]:
embedding = OllamaEmbeddings(model="mistral:v0.3").embed_query("cat")

vectorstore.similarity_search_by_vector(embedding)

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})]

## Retrievers

LangChain `VectorStore` objects do not subclass [Runnable](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.runnables) and can not immediately be integrated into LangChain Expression Language (LCEL) [chains](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel).

LangChain [Retrievers](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.retrievers) *are* Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous `invoke` and `batch` operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing `Retriever`. If we choose what method we wish to use to retrieve documents we can create a runnable easily. Below we will build one around the `similarity_search` method:

In [16]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) #selects the best result

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'})]]

**Note**: The model for some reason thinks rabbits are the closest match to shark, when it should be returning fish. Could be the model or how the data is vectorized. This is a good topic for further exploration later. Could also be the vector search method?

Vectorstores implement an `as_retriever` method that will generate a Retriever, specifically a [VectorStoreRetriever](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStoreRetriever.html). These retrivers include specific `search_type` and `search_kwargs` attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance we can replicate the above with the following:

In [17]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'})]]

`VectorStoreRetriever` supports search types of `"similarity"` (default), `"mmr"` (maximum marginal relevance, described above), and `"similarity_score_threshold"`. We can use the latter to threshold documents output by the retriever by similarity score. 

Retrievers can easily be incorporated into more complex applications, such as RAG applications that combine a given question with retrieved context into a prompt for a LLM. Below we show a minimal example.

In [18]:
# setup the model

from langchain_community.chat_models import ChatOllama
model = ChatOllama(model="mistral:v0.3", base_url="http://localhost:11434")

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model

In [20]:
response = rag_chain.invoke("Tell me about cats")

print(response.content)

 According to the document 'mammal-pets-doc', cats are considered independent pets. They tend to appreciate having their own personal space.


### Learn more:

Retrieval strategies can be rich and complex. For example:

- We can [infer hard rules and filters](https://python.langchain.com/v0.2/docs/how_to/self_query/) from a query (e.g., "using documents published after 2020");
- We can [return documents that are linked](https://python.langchain.com/v0.2/docs/how_to/parent_document_retriever/) to the retrieved context in some way (e.g., via some document taxonomy);
- We can generate [multiple embeddings](https://python.langchain.com/v0.2/docs/how_to/multi_vector/) for each unit of context;
- We can [ensemble results](https://python.langchain.com/v0.2/docs/how_to/ensemble_retriever/) from multiple retrievers;
- We can assign weights to documents, e.g., to weigh [recent documents](https://python.langchain.com/v0.2/docs/how_to/time_weighted_vectorstore/) higher.

The [retrievers](https://python.langchain.com/v0.2/docs/how_to/#retrievers) section of the how-to guides covers these and other built-in retrieval strategies. It is also straightforward to extend the [BaseRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html) class in order to implement custom retrievers. See the how-to guide [here](https://python.langchain.com/v0.2/docs/how_to/custom_retriever/).